In [63]:
%load_ext autoreload
%autoreload 2

import requests
import pandas as pd
import geopandas as gpd
from thefuzz import process
from sqlalchemy import create_engine, text, inspect
#import pydeck as pdk
import json
import warnings

import plotly.graph_objects as go
import matplotlib.pyplot as plt
from zipfile import ZipFile
import io
import datetime

import os
import sys
sys.path.insert(0, os.path.abspath('..'))
from qtrees.fisbroker import get_trees
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [116]:
db_qtrees, postgres_passwd, mitte_pw, mitte_user,  data_directory = "localhost", "postgrespostgres", "BA-Mitte-Gast-2021", "ba-mitte-gast", "../../qtrees-ai-data/data/trees"

engine = create_engine(
    f"postgresql://postgres:{postgres_passwd}@{db_qtrees}:5432/qtrees"
)

with engine.connect() as con:
    train_data = pd.read_sql('select * from api.training_data', con)
    test_data = pd.read_sql('select * from api.test_data', con)

#train_data = train_data.dropna()

In [117]:
data = []
for sensor_type in [1, 2, 3]:
    for tree in train_trees.index.unique():
        tmp =  train_data.loc[(train_data.tree_id == tree) & (train_data.sensor_type == sensor_type)]
        tmp = tmp.sort_values("timestamp")
        tmp["value_lag"] = tmp["value"].shift()
        data.append(tmp)
train_data = pd.concat(data)

In [62]:
test_trees = test_data[["id", "winter", "spring", "summer", "fall", "gattung_deutsch", "standalter"]].set_index("id")
test_trees.standalter = (test_trees.standalter-test_trees.standalter.min())/(test_trees.standalter.max()-test_trees.standalter.min())
enc = OneHotEncoder(sparse=False)
tree_dummies = pd.DataFrame(enc.fit_transform(test_trees[["gattung_deutsch"]]), index=test_trees.index)
test_trees = pd.concat([test_trees[["winter", "spring", "summer", "fall", "standalter"]], tree_dummies], axis=1)

train_trees = train_data[["tree_id", "winter", "spring", "summer", "fall", "gattung_deutsch", "standalter"]].drop_duplicates().set_index("tree_id")
train_trees.standalter = (train_trees.standalter-train_trees.standalter.min())/(train_trees.standalter.max()-train_trees.standalter.min())
tree_dummies = pd.DataFrame(enc.transform(train_trees[["gattung_deutsch"]]), index=train_trees.index)
train_trees = pd.concat([train_trees[["winter", "spring", "summer", "fall", "standalter"]], tree_dummies], axis=1)

train_trees.shape, test_trees.shape

((94, 93), (801193, 93))

In [112]:
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(train_trees.values)

test_trees = test_trees.dropna()
neighbors = pd.DataFrame(neigh.kneighbors(test_trees.values,return_distance=False), index=test_trees.index, columns=["neighbor"])

IDX_MAP = dict(zip(range(len(train_trees.index)), train_trees.index))
neighbors.neighbor.map(IDX_MAP)

id
00008100:000bbafe    00008100:00281688
00008100:000bbb03    00008100:00281688
00008100:000bbb14    00008100:002a1caa
00008100:000bbb23    00008100:001592ca
00008100:000bbb4b    00008100:00158fe1
                           ...        
00008100:002daf99    00008100:00279669
00008100:002db1d2    00008100:0028d257
00008100:002db1de    00008100:001f114b
00008100:002db1e6    00008100:001f114b
00008100:002db6ae    00008100:00281688
Name: neighbor, Length: 660168, dtype: object

In [130]:
test = []
for idx, row in test_data.iterrows():
    for sensor_type in [1,2,3]:
        print(row.append(pd.Series((1, 2), index=["sensor_type", "value_lag"])))
    break

id                 00008100:000bbafe
gattung_deutsch                AHORN
standalter                      42.0
winter                           1.0
spring                      0.509281
summer                      0.375887
fall                         0.50585
rainfall_mm                      0.0
temp_mean                       -1.8
temp_max                        -0.8
sensor_type                        1
value_lag                          2
dtype: object


/var/folders/23/h_l95d7n2x1cb9yz7mkk31rc0000gn/T/ipykernel_15878/698302750.py:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  print(row.append(pd.Series((1, 2), index=["sensor_type", "value_lag"])))


In [121]:
train_data[train_data.timestamp.dt.date==datetime.date(2022, 11, 20)]

,tree_id,sensor_type,timestamp,value,winter,spring,summer,fall,gattung_deutsch,standalter,rainfall_mm,temp_max,temp_avg,value_lag
402,00008100:00287535,1,2022-11-20 00:00:00+00:00,0.000000,1.000000,1.000000,0.732523,1.000000,AHORN,3.0,0.1,1.3,-0.4,0.000000
398,00008100:00272c9e,1,2022-11-20 00:00:00+00:00,199.262446,1.000000,0.550174,0.643364,0.547029,ULME,5.0,0.1,1.3,-0.4,198.403362
371,00008100:002a8089,1,2022-11-20 00:00:00+00:00,39.076690,0.803621,0.631961,0.583925,0.629387,LINDE,2.0,0.1,1.3,-0.4,38.521330
395,00008100:001f114a,1,2022-11-20 00:00:00+00:00,252.000000,0.803621,0.713747,0.643364,0.711746,LINDE,102.0,0.1,1.3,-0.4,252.000000
383,00008100:001f114b,1,2022-11-20 00:00:00+00:00,48.651803,0.738162,0.509281,0.465046,0.505850,ROSSKASTANIE,102.0,0.1,1.3,-0.4,49.221688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,00008100:0015aaae,3,2022-11-20 00:00:00+00:00,67.978830,1.000000,0.550174,0.346168,0.547029,BIRKE,14.0,0.1,1.3,-0.4,68.845125
469,00008100:00158fe1,3,2022-11-20 00:00:00+00:00,12.760072,1.000000,0.427494,0.286728,0.423491,LINDE,62.0,0.1,1.3,-0.4,13.408967
385,00008100:0015720a,3,2022-11-20 00:00:00+00:00,99.640438,1.000000,0.959107,0.613644,1.000000,HAINBUCHE,9.0,0.1,1.3,-0.4,98.491948
382,00008100:00157209,3,2022-11-20 00:00:00+00:00,39.469183,1.000000,1.000000,0.732523,1.000000,HAINBUCHE,9.0,0.1,1.3,-0.4,38.871992
